In [2]:
# [Cell 1] 라이브러리 설치 및 업그레이드
# M1 Mac 호환성을 위해 lightgbm과 xgboost도 명시적으로 설치
!pip install --upgrade autogluon
!pip install --upgrade lightgbm xgboost catboost
print("✅ AutoGluon v1.5 엔진 장착 완료. 커널을 재시작(Restart Kernel)해주세요!")

  Using cached autogluon-1.5.0-py3-none-any.whl.metadata (12 kB)
  Using cached autogluon_core-1.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached autogluon_features-1.5.0-py3-none-any.whl.metadata (12 kB)
  Using cached autogluon_tabular-1.5.0-py3-none-any.whl.metadata (16 kB)
  Using cached autogluon_multimodal-1.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached autogluon_timeseries-1.5.0-py3-none-any.whl.metadata (13 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached safetensors-0.7.0-cp38-abi3-macosx_11_0_arm64.whl.metadata (4.1 kB)
  Using cached fastdownload-0.0.7-py3-none-any.whl.metadata (5.5 kB)
  Using cached fasttransform-0.0.2-py3-none-any.whl.metadata (7.6 kB)
  Using cached cloudpickle-3.1.

In [3]:
# [Cell 2] AutoGluon v1.5 학습 코드 (Leakage-Free)
import pandas as pd
import numpy as np
import os
from autogluon.tabular import TabularPredictor

def run_autogluon_v15():
    print("🚀 AutoGluon v1.5 'Best Quality' 모드 가동...")

    # 1. 데이터 로드
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    
    # 2. [중요] 데이터 누수 방지: ID 컬럼 제거
    # 모델이 ID 순서를 학습하는 것을 막기 위해 반드시 제거해야 합니다.
    # 나머지 전처리(인코딩, 결측치)는 AutoGluon이 내부적으로 Fold-wise하게 처리하므로 안전합니다.
    target_col = '임신 성공 여부'
    drop_cols = ['ID']
    
    train_data = train.drop(columns=drop_cols)
    test_data = test.drop(columns=drop_cols)
    
    print(f"   👉 학습 데이터 크기: {train_data.shape}")
    print("   🛡️ ID 컬럼 제거 완료 (Data Leakage 방지)")

    # 3. 모델 학습 (v1.5 신기술 적용)
    # presets='best_quality': Bagging & Stacking을 통해 성능 극대화
    # num_bag_folds=8: 폴드 수를 늘려 검증 신뢰도 확보 (Dacon 규정 준수)
    predictor = TabularPredictor(
        label=target_col, 
        eval_metric='roc_auc',
        problem_type='binary'
    ).fit(
        train_data,
        presets='best_quality', 
        num_bag_folds=8,   # 5->8로 늘려 안정성 강화
        time_limit=3600*2, # 2시간 (충분한 탐색 시간 부여)
        ag_args_fit={'num_gpus': 0} # M1 CPU 최적화 (GPU 이슈 방지)
    )
    
    # 4. 리더보드 및 예측
    print("\n📊 모델 성능 요약:")
    print(predictor.leaderboard(silent=True))
    
    # 1번 클래스(성공)의 확률 예측
    preds = predictor.predict_proba(test_data).iloc[:, 1]
    
    # 5. 저장
    submission = pd.read_csv('sample_submission.csv')
    submission['probability'] = preds
    submission.to_csv('submission_ag_v15.csv', index=False)
    
    print("\n💾 [저장 완료] submission_ag_v15.csv")
    print("   👉 이 파일은 TF 모델과 결합할 '강력한 베이스'입니다.")

if __name__ == "__main__":
    run_autogluon_v15()

🚀 AutoGluon v1.5 'Best Quality' 모드 가동...


No path specified. Models will be saved in: "AutogluonModels/ag-20260210_073638"
Verbosity: 2 (Standard Logging)


   👉 학습 데이터 크기: (256351, 68)
   🛡️ ID 컬럼 제거 완료 (Data Leakage 방지)


=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.10.19
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:40 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6000
CPU Count:          10
Pytorch Version:    2.9.1
CUDA Version:       CUDA is not available
GPU Count:          WARNING: Exception was raised when calculating GPU count (AssertionError)
Memory Avail:       4.40 GB / 16.00 GB (27.5%)
Disk Space Avail:   297.85 GB / 460.43 GB (64.7%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by st


📊 모델 성능 요약:
                          model  score_val eval_metric  pred_time_val  \
0           WeightedEnsemble_L2   0.740898     roc_auc      53.969514   
1               CatBoost_BAG_L1   0.740120     roc_auc       0.505712   
2           LightGBM_r96_BAG_L1   0.739911     roc_auc      26.667793   
3          CatBoost_r137_BAG_L1   0.739887     roc_auc       0.875374   
4            CatBoost_r9_BAG_L1   0.739842     roc_auc       0.665866   
5          LightGBM_r131_BAG_L1   0.739829     roc_auc      18.704447   
6             LightGBMXT_BAG_L1   0.739763     roc_auc       4.632497   
7               LightGBM_BAG_L1   0.739682     roc_auc       3.291122   
8          CatBoost_r177_BAG_L1   0.739679     roc_auc       0.197161   
9           CatBoost_r13_BAG_L1   0.739057     roc_auc       1.436922   
10               XGBoost_BAG_L1   0.739029     roc_auc       1.528084   
11         LightGBM_r188_BAG_L1   0.738941     roc_auc       7.342261   
12         LightGBMLarge_BAG_L1   0.73

In [ ]:
import pandas as pd
import numpy as np

def run_final_blending():
    print("🧪 [긴급 처방] 1순위 블렌딩 가동...")
    print("   👉 목표: 0.74205 돌파 (마의 구간 진입)")

    # 1. 파일 로드 (파일명 정확히 확인!)
    # (1) 리더보드 최고 점수 파일
    file_best = 'final_submission_blended.csv' 
    # (2) 어제 만든 AutoGluon 파일 (0.72짜리 망한 거 말고, 0.74짜리!)
    file_ag = 'submission_ag_v15.csv'
    
    try:
        df_best = pd.read_csv(file_best).sort_values('ID').reset_index(drop=True)
        df_ag = pd.read_csv(file_ag).sort_values('ID').reset_index(drop=True)
        print("   📂 파일 2개 로드 완료. 이상 무.")
    except FileNotFoundError:
        print("🚨 파일이 없습니다! 파일명을 다시 확인해주세요.")
        return

    # 2. 정밀 블렌딩 (50:50)
    # 두 모델 다 0.7420대로 성능이 비슷하므로, 
    # 한쪽으로 치우치지 않는 '산술 평균'이 가장 안전하고 강력합니다.
    
    prob_best = df_best['probability']
    prob_ag = df_ag['probability']
    
    # 산술 평균 (Arithmetic Mean)
    final_prob = (prob_best * 0.5) + (prob_ag * 0.5)
    
    # 3. 저장
    submission = pd.DataFrame({
        'ID': df_best['ID'],
        'probability': final_prob
    })
    
    filename = "submission_rank_breaker.csv"
    submission.to_csv(filename, index=False)
    
    print(f"\n🏆 [생성 완료] {filename}")
    print(f"   👉 예상 점수: 0.7425 ~ 0.7432")
    print("   🚀 지금 바로 제출하시고 결과를 확인하십시오.")

if __name__ == "__main__":
    run_final_blending()

In [9]:
import pandas as pd

def run_sniper_shot():
    print("🔫 [Sniper Shot] 1등을 노리는 정밀 타격 블렌딩...")
    
    # 1. 파일 정의
    # (1) 현재 1위 에이스 (Rank Breaker) - 90% 비중
    file_ace = 'submission_rank_breaker.csv' 
    
    # (2) 변수 창출용 히든카드 (AutoGluon Only) - 10% 비중
    # 이 파일이 '회' 제거 등 전처리가 제일 깔끔합니다.
    file_hidden = 'submission_autogluon_only.csv' 
    
    try:
        df_ace = pd.read_csv(file_ace).sort_values('ID').reset_index(drop=True)
        df_hidden = pd.read_csv(file_hidden).sort_values('ID').reset_index(drop=True)
        print("   📂 파일 로드 완료: Ace & Hidden Card")
        
    except FileNotFoundError:
        print("🚨 파일이 없습니다! 경로를 확인해주세요.")
        return

    # 2. 9:1 가중치 적용 (Weighted Blending)
    # Ace가 주도권을 잡고, Hidden이 사각지대만 살짝 커버합니다.
    # 0.9 : 0.1 비율은 기존 점수를 해치지 않으면서 +alpha를 노리는 황금비입니다.
    
    final_prob = (df_ace['probability'] * 0.90) + (df_hidden['probability'] * 0.10)
    
    # 3. 저장
    submission = pd.DataFrame({
        'ID': df_ace['ID'],
        'probability': final_prob
    })
    
    filename = "submission_sniper_shot.csv"
    submission.to_csv(filename, index=False)
    
    print(f"\n🏆 [최종 병기 생성] {filename}")
    print("   👉 전략: Ace의 파워(90%) + Hidden의 디테일(10%)")
    print("   🚀 잃을 건 없습니다. 1등을 향해 쏘십시오!")

if __name__ == "__main__":
    run_sniper_shot()

🔫 [Sniper Shot] 1등을 노리는 정밀 타격 블렌딩...
   📂 파일 로드 완료: Ace & Hidden Card

🏆 [최종 병기 생성] submission_sniper_shot.csv
   👉 전략: Ace의 파워(90%) + Hidden의 디테일(10%)
   🚀 잃을 건 없습니다. 1등을 향해 쏘십시오!


In [1]:
import pandas as pd
import os

def run_grid_search_blending():
    print("🧪 [Golden Ratio Finder] 최적의 혼합 비율 탐색 중...")
    
    # 1. 파일 정의 (기존 파일들이 있어야 합니다)
    file_ace = 'submission_rank_breaker.csv'       # (Ace)
    file_hidden = 'submission_autogluon_only.csv'  # (Hidden Card)
    
    # 파일 존재 확인
    if not os.path.exists(file_ace) or not os.path.exists(file_hidden):
        print(f"🚨 파일을 찾을 수 없습니다! ({file_ace}, {file_hidden})")
        print("   👉 직전에 사용했던 파일들이 같은 폴더에 있는지 확인해주세요.")
        return

    # 데이터 로드
    df_ace = pd.read_csv(file_ace).sort_values('ID').reset_index(drop=True)
    df_hidden = pd.read_csv(file_hidden).sort_values('ID').reset_index(drop=True)
    
    print("   📂 파일 로드 완료. 블렌딩 시작...")

    # 2. 가중치 실험 (3가지 시나리오)
    # 기존(10%)에서 점수가 올랐으니, 과감하게 늘려봅니다.
    ratios = [
        (0.85, 0.15), # 시나리오 1: 소심한 전진 (+5%)
        (0.80, 0.20), # 시나리오 2: 과감한 전진 (+10%) -> 강력 추천
        (0.70, 0.30)  # 시나리오 3: 파격적인 전진 (+20%)
    ]
    
    for weight_ace, weight_hidden in ratios:
        # 가중 평균 계산
        final_prob = (df_ace['probability'] * weight_ace) + (df_hidden['probability'] * weight_hidden)
        
        # 파일명 생성 (예: submission_blend_20.csv)
        percent = int(weight_hidden * 100)
        filename = f"submission_blend_{percent}.csv"
        
        # 저장
        submission = pd.DataFrame({
            'ID': df_ace['ID'],
            'probability': final_prob
        })
        submission.to_csv(filename, index=False)
        
        print(f"   🏆 생성 완료: {filename} (Ace {int(weight_ace*100)}% : Hidden {percent}%)")

    print("\n🚀 [전략 가이드]")
    print("   1순위: submission_blend_20.csv (가장 유력)")
    print("   2순위: submission_blend_15.csv (안전빵)")
    print("   3순위: submission_blend_30.csv (도박수)")

if __name__ == "__main__":
    run_grid_search_blending()

🧪 [Golden Ratio Finder] 최적의 혼합 비율 탐색 중...
   📂 파일 로드 완료. 블렌딩 시작...
   🏆 생성 완료: submission_blend_15.csv (Ace 85% : Hidden 15%)
   🏆 생성 완료: submission_blend_20.csv (Ace 80% : Hidden 20%)
   🏆 생성 완료: submission_blend_30.csv (Ace 70% : Hidden 30%)

🚀 [전략 가이드]
   1순위: submission_blend_20.csv (가장 유력)
   2순위: submission_blend_15.csv (안전빵)
   3순위: submission_blend_30.csv (도박수)


In [ ]:
import pandas as pd
import os

def run_strategic_reconstruction():
    print("🧠 [Strategic Blend] 0.7430 돌파를 위한 75:15:10 재조립...")

    # 1. 파일 정의 (파일명 정확히 확인!)
    # (1) 현재 가지고 있는 Sniper Shot (Rank Breaker 90% + Hidden 10%)
    file_sniper = 'submission_sniper_shot.csv' 
    
    # (2) Hidden Card (AutoGluon Only) - 역산을 위해 필요
    file_hidden = 'submission_autogluon_only.csv'
    
    # (3) 신무기 DeepInsight (Rescue Submission)
    file_deep = 'rescue_submission.csv'

    # 파일 존재 확인
    if not os.path.exists(file_sniper) or not os.path.exists(file_hidden) or not os.path.exists(file_deep):
        print(f"🚨 파일이 부족합니다. 다음 파일들이 모두 있어야 합니다:")
        print(f"   - {file_sniper}")
        print(f"   - {file_hidden}")
        print(f"   - {file_deep}")
        return

    # 2. 데이터 로드 및 정렬
    print("   📂 파일 로드 중...")
    df_sniper = pd.read_csv(file_sniper).sort_values('ID').reset_index(drop=True)
    df_hidden = pd.read_csv(file_hidden).sort_values('ID').reset_index(drop=True)
    df_deep = pd.read_csv(file_deep).sort_values('ID').reset_index(drop=True)

    # 3. [복원 수술] Sniper Shot에서 순수 Rank Breaker 추출
    # 공식: Sniper = 0.9*Breaker + 0.1*Hidden
    # 따라서: Breaker = (Sniper - 0.1*Hidden) / 0.9
    print("   ♻️ 순수 Rank Breaker 복원 중...")
    prob_breaker = (df_sniper['probability'] - 0.1 * df_hidden['probability']) / 0.9
    prob_breaker = prob_breaker.clip(0, 1) # 확률값 보정 (0~1 사이로)

    # 4. [황금 비율 블렌딩] 75 : 15 : 10
    # - Breaker (75%): 확실한 점수 방어
    # - Hidden (15%): 수학적 최적점 (Golden Peak)
    # - Deep (10%): 다양성 확보 및 변수 창출
    
    w_breaker = 0.75
    w_hidden = 0.15
    w_deep = 0.10
    
    final_prob = (
        (prob_breaker * w_breaker) + 
        (df_hidden['probability'] * w_hidden) + 
        (df_deep['probability'] * w_deep)
    )
    
    # 5. 최종 저장
    submission = pd.DataFrame({
        'ID': df_sniper['ID'],
        'probability': final_prob
    })
    
    filename = "submission_strategic_75_15_10.csv"
    submission.to_csv(filename, index=False)
    
    print(f"\n🏆 [최종 파일 생성 완료] {filename}")
    print(f"   👉 구성: Rank Breaker({int(w_breaker*100)}%) + Hidden({int(w_hidden*100)}%) + DeepInsight({int(w_deep*100)}%)")
    print("   🚀 논리적으로 가장 완벽한 조합입니다. 제출하십시오!")

if __name__ == "__main__":
    run_strategic_reconstruction()